In [ ]:
# Optimization  # This is the function, I'm minimazing
def sharpe_pf(W,returns):
    pf_risk = (W.dot(returns.cov()).dot(W)) ** 0.5
    SR = W.T.dot(returns.mean()) / pf_risk
    return -SR
sharpe_pf(W,ret_df)


In [ ]:
from scipy.optimize import minimize
import numpy as np

# === Precompute ===
mu = ret_df.mean().values
cov = ret_df.cov().values
n_assets = ret_df.shape[1]

# === Objective Function ===
def neg_sharpe_lbfgsb(W, mu, cov):
    W = W / np.sum(W)  # normalize to sum to 1
    risk = np.sqrt(W @ cov @ W)
    if risk == 0:
        return np.inf
    return - (W @ mu) / risk

# === Initial Setup ===
initial_weights = np.ones(n_assets) / n_assets
bounds = [(0, 1)] * n_assets

# === Run Optimizer ===
result = minimize(
    neg_sharpe_lbfgsb,
    initial_weights,
    args=(mu, cov),
    method='L-BFGS-B',
    bounds=bounds,
    options={
        'disp': True,
        'maxiter': 1000,
        'maxfun': 50000
    }
)

# === Results ===
if result.success:
    weights_opt = result.x / np.sum(result.x)
    max_sharpe = -neg_sharpe_lbfgsb(weights_opt, mu, cov)

    print("\n✅ Optimization successful!")
    print(f"Optimization terminated successfully    (Exit mode {result.status})")
    print(f"Current function value: {result.fun}")
    print(f"Iterations: {result.nit}")
    print(f"Function evaluations: {result.nfev}")
    print(f"Gradient evaluations: {result.njev}\n")

    print("Max Sharpe Ratio:", max_sharpe)
    print("Optimal Weights:\n")
    for ticker, weight in zip(ret_df.columns, weights_opt):
        if weight > 0:
            print(f"{ticker}: {weight:.4f}")
else:
    print("\n❌ Optimization failed:", result.message)


In [ ]:
from scipy.optimize import minimize
import numpy as np

# Precompute for speed
mu = ret_df.mean().values
cov = ret_df.cov().values
n_assets = ret_df.shape[1]

# Objective function with normalization inside
def neg_sharpe_lbfgsb(W, mu, cov):
    W = W / np.sum(W)  # normalize weights to sum to 1
    risk = np.sqrt(W @ cov @ W)
    if risk == 0:
        return np.inf
    return - (W @ mu) / risk

# Initial guess
initial_weights = np.ones(n_assets) / n_assets

# Bounds (no short selling)
bounds = [(0, 1)] * n_assets

# Run optimizer
result = minimize(
    neg_sharpe_lbfgsb,
    initial_weights,
    args=(mu, cov),
    method='L-BFGS-B',
    bounds=bounds,
    options={'disp': False, 'maxiter': 300}
)

# Normalize final weights
weights_opt = result.x / np.sum(result.x)

# Output
print("\n⚡️ FAST Optimization Results:")
print(f"Optimization terminated: {result.message} (Exit mode {result.status})")
print(f"Current function value: {result.fun}")
print(f"Iterations: {result.nit}")
print(f"Function evaluations: {result.nfev}")
print(f"Gradient evaluations: {result.njev}\n")

print("Max Sharpe Ratio:", -(neg_sharpe_lbfgsb(weights_opt, mu, cov)))
print("Optimal Weights:")
for ticker, weight in zip(ret_df.columns, weights_opt):
    if weight > 0:
        print(f"{ticker}: {weight:.4f}")

In [ ]:
# === STEP 1: Sharpe ratio function (negative, because we minimize) ===
def neg_sharpe_ratio(weights, returns):
    portfolio_returns = returns @ weights  # (T,)
    mean_return = np.mean(portfolio_returns)
    std_return = np.std(portfolio_returns)
    if std_return == 0:
        return np.inf  # avoid division by zero
    return -mean_return / std_return

# === STEP 2: Constraints and bounds ===
n_assets = ret_df.shape[1]

# Constraint: sum of weights = 1
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# Bounds: no short-selling (0 ≤ w ≤ 1)
bounds = [(0, 1)] * n_assets

# Initial guess: equal weighting
initial_weights = np.ones(n_assets) / n_assets

# === STEP 3: Optimization ===
res = minimize(
    neg_sharpe_ratio,
    initial_weights,
    args=(ret_df,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'maxiter': 1000}
)

# === STEP 4: Results ===
if res.success:
    print("\n✅ Optimization successful!")
    print("Max Sharpe Ratio:", -res.fun)
    print("Optimal Weights:\n")
    for ticker, weight in zip(ret_df.columns, res.x):
        print(f"{ticker}: {weight:.4f}")
else:
    print("\n❌ Optimization failed:", res.message)


In [ ]:
for ticker, weight in zip(ret_df.columns, res.x):
    if weight > 0.001:
        print(f"{ticker}: {weight:.4f}")


In [ ]:
res

In [ ]:
opt_W = res['x']

In [ ]:
opt_W

In [ ]:
# Return for optimal weigth
ret_df.dot(opt_W).cumsum()

In [ ]:
# Return for equal weigth
ret_df.dot(W).cumsum()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# === Step 1: Compute cumulative portfolio return (linear, not log) ===
portfolio_returns = ret_df.dot(opt_W)
cumulative_return = portfolio_returns.cumsum()  # No np.exp!

# === Step 2: Create interactive plot ===
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ret_df.index,
    y=cumulative_return,
    mode='lines',
    name='Optimized Portfolio',
    line=dict(width=2, color='green')
))

# === Step 3: Customize layout ===
fig.update_layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    width=1000,
    height=600,
    template="plotly_white"
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# === Step 1: Compute cumulative portfolio return (linear, not log) ===
portfolio_returns = ret_df.dot(opt_W)
cumulative_return = portfolio_returns.cumsum()  # No np.exp!

# === Step 2: Create interactive plot ===
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ret_df.index,
    y=cumulative_return,
    mode='lines',
    name='Optimized Portfolio',
    line=dict(width=2, color='green')
))

# === Step 3: Customize layout ===
fig.update_layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    width=1000,
    height=600,
    template="plotly_white"
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# === Step 1: Compute cumulative portfolio return ===
portfolio_returns = ret_df.dot(opt_W)
cumulative_return = np.exp(portfolio_returns.cumsum())  # $1 growth over time

# === Step 2: Create interactive plot ===
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ret_df.index,
    y=cumulative_return,
    mode='lines',
    name='Optimized Portfolio',
    line=dict(width=2, color='green')
))

# === Step 3: Customize layout ===
fig.update_layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Portfolio Value ($1 Start)",
    width=1000,
    height=600,
    template="plotly_white"
)

fig.show()


In [ ]:
len(ret_df) * 0.6

In [ ]:
train = ret_df[:int(len(ret_df) * 0.6)]

In [ ]:
train

In [ ]:
test = ret_df[int(len(ret_df) * 0.6):]

In [ ]:
test

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# === Step 0: Select Top 550 Most Volatile Assets ===
top_550_tickers = train.std().nlargest(550).index
train_reduced = train[top_550_tickers]

# === Step 1: Precompute everything ===
mu = train_reduced.mean().to_numpy()
cov = train_reduced.cov().to_numpy()
n_assets = len(mu)

# === Step 2: Define fast Sharpe function ===
def neg_sharpe(weights, mu, cov):
    weights = weights / np.sum(weights)  # Normalize
    port_return = weights @ mu
    port_std = np.sqrt(weights @ cov @ weights)
    return np.inf if port_std == 0 else -port_return / port_std

# === Step 3: Optimization Setup ===
W0 = np.ones(n_assets) / n_assets
bounds = [(0, 1)] * n_assets

# === Step 4: Run optimizer ===
res = minimize(
    neg_sharpe,
    W0,
    args=(mu, cov),
    method='L-BFGS-B',
    bounds=bounds,
    options={
        'disp': True,
        'maxiter': 1000,
        'maxfun': 200000  # More evaluations for big problems
    }
)

# === Step 5: Results ===
if res.success:
    weights_opt = res.x / np.sum(res.x)
    max_sharpe = -neg_sharpe(weights_opt, mu, cov)

    print("\n✅ Optimization Successful!")
    print(f"Max Sharpe Ratio (Train) : {max_sharpe:.6f}")
    print(f"Iterations: {res.nit}")
    print(f"Function evaluations: {res.nfev}")
    print(f"Gradient evaluations: {res.njev}\n")
    print("Top Portfolio Weights (non-zero):\n")
    for ticker, weight in zip(train_reduced.columns, weights_opt):
        if weight > 1e-4:
            print(f"{ticker}: {weight:.4f}")
else:
    print("\n❌ Optimization Failed:", res.message)


In [ ]:
res

In [ ]:
#test.dot(res_train.x).cumsum()
# Ensure test data uses the same 100 assets as train_reduced
test_reduced = test[train_reduced.columns]

# Now it will work
cumulative_return = test_reduced.dot(res.x).cumsum()

# Optional: convert log returns to price
portfolio_value = np.exp(cumulative_return)

portfolio_value

In [ ]:
test.dot(W).cumsum()

In [ ]:
# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test_reduced.index, portfolio_value, linewidth=2)
plt.title("Optimized Portfolio Cumulative Return on Test Data")
plt.xlabel("Date")
plt.ylabel("Portfolio Value ($1 Start)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# === Step 1: Compute cumulative portfolio value from log returns ===
# Assuming test_reduced and res_train.x are defined
portfolio_log_returns = test_reduced.dot(res.x)
portfolio_value = np.exp(portfolio_log_returns.cumsum())  # Convert log return to price

# === Step 2: Create interactive plot ===
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=test_reduced.index,
    y=portfolio_value,
    mode='lines',
    name='Optimized Portfolio',
    line=dict(width=2, color='green')
))

# === Step 3: Customize layout ===
fig.update_layout(
    title="Optimized Portfolio Cumulative Return on Test Data",
    xaxis_title="Date",
    yaxis_title="Portfolio Value ($1 Start)",
    width=1000,
    height=600,
    template="plotly_white"
)

fig.show()


In [ ]:
sharpe_pf(res.x, test_reduced)

In [ ]:
sharpe_pf(W,test_reduced)